<a href="https://colab.research.google.com/github/bosswin/jupyter-note/blob/master/VectorDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess
import glob, os
model_dir = "iic/SenseVoiceSmall"

results =[]
model = AutoModel(
    model=model_dir,
    vad_model="fsmn-vad",
    vad_kwargs={"max_single_segment_time": 30000,"hotword":"法属圣马丁"},
    hotwords="法属圣马丁"
    )
os.chdir("/content/team_wav") # 切换到指定目录


language_maps={"p":"zh","e":"en","c":"yue"}


res = model.generate(
    input=f"/content/team_wav/pFSM.vox.wav",
    cache={},
    language="zh",  # "zh", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
    hotword="法属圣马丁"
)
text = rich_transcription_postprocess(res[0]["text"])
text

In [ ]:
import IPython.display as dp
import pandas as pd
df = pd.DataFrame(results)
dp.display(df)

In [ ]:
first = "eNM2.vox.wav"[0]

language_map={"p":"zh","e":"en","c":"yue"}
language_map["p"]

In [ ]:
!pip install zhconv

In [ ]:
import zhconv
import time
time1 = time.time()
s='''聖安東尼奧聯
康塞普森體育
利納雷斯體育
列高列達
利馬切
特拉桑迪洛
上普恩特
彼達迪華馬卡比
伊比利亞1999
奧克蘭FC
AC維圖斯
克里維里赫
帕福斯
胡內多阿拉
普萊西亞日托米爾
薩姆加耶特
FK特蘭斯
NK巴禾
馬利舒華
FA希奧利艾
FC諾亞
FC伊斯洛
施特拉森
塔林卡列夫
卡納芬
迪華殊
莫爾納
丹達
烏爾姆
雷尼基1923
皇家聖祖亞昆
日本足球學院
阿富汗U20
土庫曼U20
美國男奧足
多明尼加共和國男奧足
以色列男奧足
烏克蘭男奧足
馬里男奧足
巴拉圭男奧足
烏茲別克男奧足
畿內亞男奧足
牙買加U20
海地U20
捷克女足U23
洛克達爾以利亞日
摩頓城精英隊
達爾文之心
奧康納騎士
珀斯紅星
蘭布頓查法斯
墨爾本塞爾維亞
格林諾治騎士
奧林匹克京士威
奧地利女足U20
拉巴斯體育會
莫雷利亞體育會
坎昆FC
塔巴迪奧
特帕蒂特蘭
特拉斯卡拉
卡拉雷斯
曼托華
卡塔爾SC
艾沙瑪
多哈艾阿里
艾科爾
艾沙哈尼亞
艾侯賽因伊比
拉夫尚庫洛布
艾卡列迪亞
東賓高
艾阿里麥納麥
阿恩艾沙
宿霧FC
南定
清化
雪蘭莪
斐濟女足U20
伊斯韋斯
伊斯韋斯
卡赫利
艾科魯特
艾科多
艾奧魯巴
艾卡迪沙
艾拉德
艾利雅德
達馬克FC
哈斯勒荷姆
車里雅賓斯克
莫斯科切達奴禾
帝國聯
特克斯和凱科斯群島
史托辛特
法屬聖馬丁
科斯特羅馬斯巴達
派迪高斯卡
安哥拉U20
博茲瓦納U20
科摩羅U20
伊斯瓦蒂尼U20
萊索托U20
馬拉維U20
納米比亞U20
津巴布韋U20
馬達加斯加U20
'''
s2=zhconv.convert(s,'zh-cn')
time2 = time.time()
cost_time = (time2 - time1)
print(s2)
print(cost_time)

In [ ]:
!pip install -U funasr modelscope

In [ ]:
from modelscope.hub.snapshot_download import snapshot_download
model_dir = snapshot_download('damo/nlp_bart_text-error-correction_chinese')

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

model_id = 'damo/nlp_bart_text-error-correction_chinese'
input = '这洋的话，下一年的福气来到自己身上。'
pipeline = pipeline(Tasks.text_error_correction, model=model_id)
result = pipeline(input)
print(result['output'])

In [ ]:
!pip install chromadb

In [4]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")


In [7]:
collection.add(
    documents=[
        '''{"code":"DLS","e":"Deportes Linares","p":"利納雷斯體育","c":"利纳雷斯体育"}''',
        '''{"code":"PZR","e":"Polissya Zhytomyr","c":"普萊西亞日托米爾","p":"普莱西亚日托米尔"}'''
    ],
    ids=["DLS", "PZR"]
)



In [12]:
results = collection.query(
    query_texts=["aeportes "], # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)

{'ids': [['DLS']], 'embeddings': None, 'documents': [['{"code":"DLS","e":"Deportes Linares","p":"利納雷斯體育","c":"利纳雷斯体育"}']], 'uris': None, 'data': None, 'metadatas': [[None]], 'distances': [[1.606644868850708]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [ ]:
!pip install -U pymilvus

In [85]:
from pymilvus import MilvusClient, DataType

# 实例化客户端，连接 Milvus 服务
client = MilvusClient("milvus_demo1.db")
# 创建 schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)
# schema 添加字段 id、vector
schema.add_field(field_name="code", datatype=DataType.VARCHAR,max_length=50, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=768)
schema.add_field(field_name="value", datatype=DataType.VARCHAR,max_length=5000)
# 创建集合
client.create_collection(
  collection_name="demo_v4",
  schema=schema,
)


DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 850773cf1d1d466d83560c264ef4107b
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: demo_v4


In [ ]:
!pip install "pymilvus[model]"

In [112]:
from pymilvus.model.hybrid import MGTEEmbeddingFunction

ef = MGTEEmbeddingFunction(
    model_name="Alibaba-NLP/gte-multilingual-base", # Defaults to `Alibaba-NLP/gte-multilingual-base`
)
print("Dim:", ef.dim )
dense_dim = ef.dim["dense"]

Dim: {'dense': 768, 'sparse': 250002}


In [104]:
docs = [
    '''{"code":"DLS","e":"Deportes Linares","p":"利納雷斯體育","c":"利纳雷斯体育"}''',
    '''{"code":"PZR","e":"Polissya Zhytomyr","c":"普萊西亞日托米爾","p":"普莱西亚日托米尔"}'''
]

vectors = ef.encode_documents(docs)["dense"]

data = [
    {"code": "DLS", "vector": vectors[0], "value": docs[0]},
    {"code": "PZR", "vector": vectors[1], "value": docs[1]}
]


In [105]:
len(vectors[0])

768

In [106]:
client.insert(collection_name="demo_v4", data=data)

{'insert_count': 2, 'ids': ['DLS', 'PZR'], 'cost': 0}

In [ ]:
query_vectors = ef.encode_queries(["体育"])

res = client.search(
    collection_name="demo_v4",  # target collection
    data=query_vectors,  # query vectors
    limit=5,  # number of returned entities
    output_fields=["code"],  # specifies fields to be returned
)

In [109]:
res = client.list_indexes(collection_name="demo_v4")

print(res)

[]


In [ ]:
result = search(query="体育")

In [60]:
import pandas as pd

In [61]:
df = pd.read_csv('team.csv')
df.head()

,Code,E,C,P
0,SAU,San Antonio Unido,聖安東尼奧聯,圣安东尼奥联
1,DCC,Deportes Concepcion,康塞普森體育,康塞普森体育
2,DLS,Deportes Linares,利納雷斯體育,利纳雷斯体育
3,DRL,Deportes Recoleta,列高列達,列高列达
4,DLE,Deportes Limache,利馬切,利马切


In [122]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

# Connect to Milvus given URI
connections.connect(uri="./milvus.db")

# Specify the data schema for the new Collection
fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="code", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim)
]
schema = CollectionSchema(fields)

# Create collection (drop the old one if exists)
col_name = "hybrid_demo"
if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")

# To make vector search efficient, we need to create indices for the vector fields
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()

In [119]:
import pandas as pd

file_path = "/content/team.csv"
df = pd.read_csv(file_path, sep=",")
data = [
     {
          "code": df["Code"][i],
          "e": df["E"][i],
          "sparse_vector": df["title"][i],
          "dense_vector": df["artist"][i],
     }
     for i in range(len(df))
]

In [120]:
docs

['特克斯和凯科斯群岛',
 '马利舒华',
 '莱索托U20',
 '曼托华',
 '上普恩特',
 '安哥拉U20',
 '艾拉德',
 '达马克FC',
 '艾沙玛',
 '土库曼U20',
 '格林诺治骑士',
 '南定',
 'FA希奥利艾',
 '迪华殊',
 '日本足球学院',
 '胡内多阿拉',
 '清化',
 '雪兰莪',
 '科斯特罗马斯巴达',
 '艾卡迪沙',
 '珀斯红星',
 '奥地利女足U20',
 '奥康纳骑士',
 '艾沙哈尼亚',
 '捷克女足U23',
 '克里维里赫',
 '萨姆加耶特',
 '拉巴斯体育会',
 '摩顿城精英队',
 '伊比利亚1999',
 '莫尔纳',
 '美国男奥足',
 'FC诺亚',
 '牙买加U20',
 '艾利雅德',
 '艾阿里麦纳麦',
 '洛克达尔以利亚日',
 '利马切',
 '津巴布韦U20',
 '阿恩艾沙',
 '丹达',
 '多米尼加共和国男奥足',
 '阿富汗U20',
 '艾科多',
 '兰布顿查法斯',
 '特拉斯卡拉',
 '雷尼基1923',
 '莫雷利亚体育会',
 '皇家圣祖亚昆',
 '塔巴迪奥',
 '卡纳芬',
 '艾卡列迪亚',
 '乌尔姆',
 '艾科尔',
 '哈斯勒荷姆',
 '列高列达',
 '坎昆FC',
 '施特拉森',
 '以色列男奥足',
 '特帕蒂特兰',
 '宿务FC',
 '乌克兰男奥足',
 '奥林匹克京士威',
 '伊斯瓦蒂尼U20',
 '康塞普森体育',
 '帕福斯',
 '帝国联',
 '伊斯韦斯',
 '马里男奥足',
 '乌兹别克男奥足',
 '几内亚男奥足',
 '达尔文之心',
 '车里雅宾斯克',
 'NK巴禾',
 '马达加斯加U20',
 '巴拉圭男奥足',
 '圣安东尼奥联',
 '利纳雷斯体育',
 '马拉维U20',
 '艾侯赛因伊比',
 '史托辛特',
 '卡拉雷斯',
 '艾科鲁特',
 '博兹瓦纳U20',
 'AC维图斯',
 '塔林卡列夫',
 '东宾高',
 '科摩罗U20',
 '卡赫利',
 '派迪高斯卡',
 '纳米比亚U20',
 'FK特兰斯',
 '莫斯科切达奴禾',
 '普莱西亚日托米尔',
 '斐济女足U20',
 '多哈艾阿里',
 '法属圣马丁',
 '特拉桑迪洛',
 '